In [1]:
import pandas as pd
import sqlite3

In [2]:
connect=sqlite3.connect('../data/checking-logs.sqlite', check_same_thread=False)

In [3]:
query = '''select time, avg(diff) as avg_diff
from (select uid, cast((deadlines.deadlines -strftime('%s', test.first_commit_ts))/3600 as integer) as diff,
case when test.first_commit_ts < test.first_view_ts then 'before'
else 'after' end as time
from test left join deadlines on test.labname = deadlines.labs where labname <> 'project1')
where uid in (select uid from (select uid,
                           case when test.first_commit_ts < test.first_view_ts then 'before'
                           else 'after' end as time
                           from test left join deadlines on test.labname=deadlines.labs where labname <> 'project1')
group by uid having count(distinct time)=2) group by time'''
test_results = pd.io.sql.read_sql(query, connect)
test_results

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [5]:
query = '''select time, avg(diff) as avg_diff
from (select uid, cast((deadlines.deadlines -strftime('%s', control.first_commit_ts))/3600 as integer) as diff,
case when control.first_commit_ts < control.first_view_ts then 'before'
else 'after' end as time
from control left join deadlines on control.labname = deadlines.labs where labname <> 'project1')
where uid in (select uid from (select uid,
                           case when control.first_commit_ts < control.first_view_ts then 'before'
                           else 'after' end as time
                           from control left join deadlines on control.labname=deadlines.labs where labname <> 'project1')
group by uid having count(distinct time)=2) group by time'''
control_results = pd.io.sql.read_sql(query, connect)
control_results

,time,avg_diff
0,after,117.636364
1,before,99.464286


In [6]:
connect.close()